### Vanilla GAN

* generator

10 samples from U(0, 1) -> 128 -> relu -> 256 -> relu -> 784 -> sigm -> 28x28

* discriminator

28x28 from data -> flatten to 784 -> 784 -> relu -> 128 -> relu -> 128 -> sigm

In [ ]:
import random

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(X, y), _ = tf.keras.datasets.fashion_mnist.load_data()
X = X / 128.0

In [ ]:
(lambda cls, i: plt.imshow(
    X[y == cls][i].reshape((28, 28)), cmap='gray'))(3, 10)

In [ ]:
fmnist = X[y == 3][:1000].reshape([-1, 28, 28, 1])

In [ ]:
def generator(z):
    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
        h1 = tf.layers.dense(z, units=128,
                            activation=tf.nn.relu)
        h2 = tf.layers.dense(h1, units=256,
                            activation=tf.nn.relu)
        prob = tf.layers.dense(h2, units=784,
                            activation=tf.nn.sigmoid)
    return tf.reshape(prob, shape=(-1, 28, 28, 1))

In [ ]:
def discriminator(x):
    with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
        flat = tf.contrib.layers.flatten(x)
        h1 = tf.layers.dense(flat, units=784,
                            activation=tf.nn.relu)
        h2 = tf.layers.dense(h1, units=128,
                            activation=tf.nn.relu)
        prob = tf.layers.dense(h2, units=128,
                            activation=tf.nn.sigmoid)
    return prob

In [ ]:
z = tf.placeholder(
    tf.float32, shape=(None, 10), name='z')
x = tf.placeholder(
    tf.float32, shape=(None, 28, 28, 1), name='x')

In [ ]:
sample = generator(z)
dreal = discriminator(x)
dfake = discriminator(sample)

In [ ]:
dloss = -tf.reduce_mean(tf.log(dreal) + tf.log(1. - dfake))
gloss = -tf.reduce_mean(tf.log(dfake))

In [ ]:
dopt = tf.train.AdamOptimizer(0.001).minimize(dloss)
gopt = tf.train.AdamOptimizer(0.001).minimize(gloss)

In [ ]:
def sample_z(m, n):
    return np.random.uniform(0., 1., size=(m, n))

In [ ]:
sess = tf.Session()
epochs = 10
batch_size = 200
sess.run(tf.global_variables_initializer())

try:
    for e in range(epochs):
        print(f"Epoch: {e+1}/{epochs}")
        for i in range(len(fmnist) // batch_size):
            batch = fmnist[random.sample(range(len(fmnist)), batch_size)]
            z_noise = sample_z(batch_size, 10)
            _, dlossc = sess.run([dopt, dloss], feed_dict={x: batch, z: z_noise})
            _, glossc = sess.run([gopt, gloss], feed_dict={z: z_noise})            
            if i % 20 == 0:
                print(f"iter: {i}, dloss: {str(round(dlossc, 2))}, gloss: {str(round(glossc, 2))}")
except KeyboardInterrupt:
    print("Stopped")

### Sample latent space

In [ ]:
latent = np.random.normal(0., 1., 10).reshape((1, 10))
decoded = sess.run(sample, feed_dict={z: latent})
plt.imshow(decoded.reshape((28, 28)), cmap='gray')

In [ ]:
sess.close()